<h1>Train GNN<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#HotpotQA_Dataset_New" data-toc-modified-id="HotpotQA_Dataset_New-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>HotpotQA_Dataset_New</a></span></li><li><span><a href="#GNN" data-toc-modified-id="GNN-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GNN</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>训练</a></span><ul class="toc-item"><li><span><a href="#超参数" data-toc-modified-id="超参数-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>超参数</a></span></li><li><span><a href="#辅助函数" data-toc-modified-id="辅助函数-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>辅助函数</a></span></li><li><span><a href="#实例化" data-toc-modified-id="实例化-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>实例化</a></span></li><li><span><a href="#开始训练" data-toc-modified-id="开始训练-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>开始训练</a></span></li></ul></li><li><span><a href="#保存与加载" data-toc-modified-id="保存与加载-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>保存与加载</a></span></li><li><span><a href="#阶段2" data-toc-modified-id="阶段2-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>阶段2</a></span><ul class="toc-item"><li><span><a href="#find-top-K-sents" data-toc-modified-id="find-top-K-sents-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>find top K sents</a></span></li><li><span><a href="#测试find-span" data-toc-modified-id="测试find-span-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>测试find span</a></span></li><li><span><a href="#计算span" data-toc-modified-id="计算span-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>计算span</a></span></li></ul></li><li><span><a href="#top-K-sent-acc" data-toc-modified-id="top-K-sent-acc-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>top K sent acc</a></span></li></ul></div>

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/folders/')
    !pip install transformers
    # !pip install -U spacy[cuda100]
    # !wget -P /content/folders/My\ Drive/download/ https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz
    # !pip install /content/folders/My\ Drive/download/en_core_web_lg-2.2.5.tar.gz
    # !wget -P /content/folders/My\ Drive/HotpotQA/ http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
    # json_train_path = '/content/folders/My Drive/HotpotQA/样例_hotpot_train_v1.1.json' # 5个例子
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
    save_cache_path = '/content/folders/My Drive/save_cache/'
    save_cache_path_linux = '/content/folders/My\ Drive/save_cache/'
    HotpotQA_path = '/content/folders/My Drive/HotpotQA'
except:
    json_train_path = r'./data/hotpot_train_v1.1.json'
    HotpotQA_path = './'
    save_cache_path = 'save_cache/'
    use_proxy = False
    proxies={"http_proxy": "127.0.0.1:10809",
        "https_proxy": "127.0.0.1:10809"} if use_proxy else None


In [2]:
import os
import sys
from argparse import Namespace
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import scipy.sparse as sp
import ujson as json
from tqdm.notebook import tqdm
from transformers import AutoConfig, AutoModel, AutoTokenizer
from apex import amp

sys.path.insert(0,HotpotQA_path)

# HotpotQA_Dataset_New

In [3]:
from datasets import HotpotQA_GNN_Dataset, gen_GNN_batches

# GNN

In [4]:
from GNN import GAT_HotpotQA

# 训练

## 超参数

In [5]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
        
args = Namespace(
    # Data and path information
    json_train_path=r'./data/hotpot_train_v1.1.json',
    json_train_mini_path=r'./data/hotpot_train_mini.json',
    model_state_file = "GNN_HotpotQA.pt",
    save_dir = 'save_cache',
    hotpotQA_item_folder = 'save_preprocess_new',
    model_path = '/g/data/models/xlnet-large-cased',
    use_proxy = False,
    proxies={"http_proxy": "127.0.0.1:10809",
             "https_proxy": "127.0.0.1:10809"},

    # Dataset parameter
    pad_max_num = 450,
    pad_value = 0,

    # Training hyper parameter
    chunk_size = 500,
    num_epochs=5,
    learning_rate=1e-3,
    batch_size=20,
    seed=1337,
    early_stopping_criteria=5,
    flush_secs=60,
    
    # Runtime hyper parameter
    cuda=True,
    device=None,
    tpu=False,
    catch_keyboard_interrupt=True,
    reload_from_files=True,
    expand_filepaths_to_save_dir=True,
    
    )

In [6]:
if not torch.cuda.is_available():
    args.cuda = False

if not args.device:
    args.device = torch.device("cuda" if args.cuda else "cpu")

if args.expand_filepaths_to_save_dir:
    args.model_state_file = os.path.join(args.save_dir,args.model_state_file)
#     args.HotpotQA_preprocess_file = os.path.join(args.save_dir,args.HotpotQA_preprocess_file)

set_seed_everywhere(args.seed, args.cuda)

handle_dirs(args.save_dir)

print("Using: {}".format(args.device))

total_items = !ls -l $args.hotpotQA_item_folder |grep "^-"| wc -l
total_items = int(total_items[0])

print(f"total items: {total_items}")

Using: cuda
total items: 50000


## 辅助函数

In [35]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            
            'train_running_loss': [],
            'train_running_recall_sent': [],
            'train_running_recall_para': [],
            'train_running_acc_Qtype': [],
            
            'val_running_loss': [],
            'val_running_recall_sent': [],
            'val_running_recall_para': [],
            'val_running_acc_Qtype': [],

            'test_running_loss': [],
            'test_running_recall_sent': [],
            'test_running_recall_para': [],
            'test_running_acc_Qtype': [],
            
            'model_filename': args.model_state_file}

def update_train_state(args, model, optimizer, train_state):
    # Save one model at least
    if train_state['epoch_index'] == 0:
        checkpoint = {
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'amp': amp.state_dict()
        }
        torch.save(checkpoint, train_state['model_filename'])
        
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_running_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                
                checkpoint = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'amp': amp.state_dict()
                }
                torch.save(checkpoint, train_state['model_filename'])

                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(logits, labels):
    _, logits_indices = logits.max(dim=1)
    n_correct = torch.eq(logits_indices, labels).sum().item()
    return float(n_correct) / len(logits_indices)

def compute_recall(logits, labels, mask):
    '''only count the positive recall'''
    _, logits_indices = logits.max(dim=1)
    all_positive_predicts = (logits_indices * labels * mask).sum().item() # only positive
    all_positive_labels = (labels * mask).sum().item()
    return float(all_positive_predicts) / all_positive_labels

In [82]:
logits = torch.tensor([
                [0.1, 0.99],
                [0.6, 0.4],
                [0.1, 0.91],
                [0.6, 0.4],
                [0.1, 0.92]
            ])
logits_B = torch.tensor([[
                [0.1, 0.99],
                [0.6, 0.4],
                [0.1, 0.91],
                [0.6, 0.4],
                [0.1, 0.92]
            ],
            [
                [1.1, 0.99],
                [1.6, 0.4],
                [0.1, 0.91],
                [0.6, 0.4],
                [1.1, 0.92]
            ]])
labels = torch.tensor([[1 ,0, 0, 1, 0],[1 ,0, 0, 1, 0]])
m0ask = torch.tensor([[1 ,1, 0, 0, 1],[1 ,1, 0, 0, 1]])
p0redict = torch.tensor([[1 ,1, 1, 1, 0],[1 ,1, 1, 1, 0]])

index_test = torch.tensor([[0, 4, 1, 3],[1, 2, 3, 4]])
compute_recall(logits, labels, mask)

1.0

In [84]:
value, indexs = logits_B.max(dim=-1)
value

tensor([[0.9900, 0.6000, 0.9100, 0.6000, 0.9200],
        [1.1000, 1.6000, 0.9100, 0.6000, 1.1000]])

In [87]:
value*mask

tensor([[0.9900, 0.6000, 0.0000, 0.0000, 0.9200],
        [1.1000, 1.6000, 0.0000, 0.0000, 1.1000]])

In [86]:
values2,indexs2 = (value*mask).topk(4, dim=-1)
indexs2

tensor([[0, 4, 1, 3],
        [1, 4, 0, 3]])

In [72]:
torch.index_select(predict, -1, indexs2)

tensor([1, 0, 1, 1])

In [73]:
torch.index_select((labels*mask), -1, indexs2)

tensor([1, 0, 0, 0])

In [81]:
torch.gather(logits_B.max(dim=-1)[1], -1, index_test)

tensor([[1, 1, 0, 0],
        [0, 1, 0, 0]])

In [91]:
max_value, max_index = logits_B.max(dim=-1) # max_index is predict class.
topN_sent_index_batch = (max_value * mask).topk(4, dim=-1)[1]
topN_sent_predict = torch.gather(max_index, -1, topN_sent_index_batch)
topN_sent_label = torch.gather(labels*mask, -1, topN_sent_index_batch)

In [92]:
topN_sent_predict

tensor([[1, 1, 0, 0],
        [0, 0, 0, 0]])

In [93]:
topN_sent_label

tensor([[1, 0, 0, 0],
        [0, 0, 1, 0]])

## 实例化

In [8]:
classifier = GAT_HotpotQA(features=768, hidden=32, nclass=2, dropout=0.2, alpha=0.3, nheads=8, nodes_num=args.pad_max_num)
classifier = classifier.to(args.device)
classifier.train()

class_weights_sent, class_weights_para, class_weights_Qtype = \
            HotpotQA_GNN_Dataset.get_weights(device=args.device)
loss_func_sent = nn.CrossEntropyLoss(class_weights_sent,ignore_index=-100)
loss_func_para = nn.CrossEntropyLoss(class_weights_para,ignore_index=-100)
loss_func_Qtype = nn.CrossEntropyLoss(class_weights_Qtype)

optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()),
                      lr=args.learning_rate)

# Initialization
opt_level = 'O1'
classifier, optimizer = amp.initialize(classifier, optimizer, opt_level=opt_level)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,\
                        mode='min', factor=0.5, patience=3)

train_state = make_train_state(args)

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [ ]:
dataset.set_split('train')
batch_generator = gen_batches(dataset,
                    batch_size=args.batch_size, 
                    device=args.device)
for batch_index, batch_dict in enumerate(batch_generator):

    optimizer.zero_grad()
    print(batch_dict)

    logits_sent, logits_para, logits_Qtype = \
                    classifier(batch_dict['feature_matrix'],batch_dict['adj'])

    logits_sent = (batch_dict['sent_mask'] * logits_sent).view(-1,2)
    labels_sent = (batch_dict['sent_mask'] * batch_dict['labels']).view(-1)

    logits_para = (batch_dict['para_mask'] * logits_para).view(-1,2)
    labels_para = (batch_dict['para_mask'] * batch_dict['labels']).view(-1)

    loss_sent = loss_func_sent(logits_sent, labels_sent) # [B,2] [B]
    loss_para = loss_func_para(logits_para, labels_para) # [B,2] [B]
    loss_Qtype = loss_func_Qtype(logits_Qtype, batch_dict['answer_type']) # [B,2] [B]

    break

## 开始训练

In [9]:
try:
    writer = SummaryWriter(flush_secs=60)
    for chunk_i in range(0, total_items, args.chunk_size):
        dataset = HotpotQA_GNN_Dataset.build_dataset(hotpotQA_item_folder = args.hotpotQA_item_folder,
                                                     i_from = chunk_i, 
                                                     i_to = chunk_i+args.chunk_size)
        dataset.set_parameters(args.pad_max_num, args.pad_value)

        epoch_bar = tqdm(desc='training routine',
                        total=args.num_epochs,
                        position=0)

        dataset.set_split('train')
        train_bar = tqdm(desc='split=train',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1)

        dataset.set_split('val')
        val_bar = tqdm(desc='split=val',
                        total=dataset.get_num_batches(args.batch_size), 
                        position=1)


        for epoch_index in range(args.num_epochs):

            train_state['epoch_index'] = epoch_index

            dataset.set_split('train')
            batch_generator = gen_GNN_batches(dataset,
                                            batch_size=args.batch_size, 
                                            device=args.device)
            running_loss = 0.0

            running_recall_sent = 0.0
            running_recall_para = 0.0
            running_acc_Qtype = 0.0

            classifier.train()


            for batch_index, batch_dict in enumerate(batch_generator):

                optimizer.zero_grad()

                logits_sent, logits_para, logits_Qtype = \
                                classifier(batch_dict['feature_matrix'], batch_dict['adj'])

                logits_sent = (batch_dict['sent_mask'] * logits_sent).view(-1,2)

                labels_sent = (batch_dict['sent_mask']*batch_dict['labels'] + \
                               batch_dict['sent_mask'].eq(0)*-100).view(-1)

                logits_para = (batch_dict['para_mask'] * logits_para).view(-1,2)

                labels_para = (batch_dict['para_mask']*batch_dict['labels'] + \
                               batch_dict['para_mask'].eq(0)*-100).view(-1)


                loss_sent = loss_func_sent(logits_sent, labels_sent) # [B,2] [B]
                loss_para = loss_func_para(logits_para, labels_para) # [B,2] [B]
                loss_Qtype = loss_func_Qtype(logits_Qtype.view(-1,2),
                                             batch_dict['answer_type'].view(-1)) # [B,2] [B]

                loss = loss_sent + loss_para + loss_Qtype
                running_loss += (loss.item() - running_loss) / (batch_index + 1)

                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()

                optimizer.step()

                # compute the recall
                recall_t_sent = compute_recall(logits_sent.view(-1,2), 
                                               batch_dict['labels'].view(-1), 
                                               batch_dict['sent_mask'].view(-1))
                running_recall_sent += (recall_t_sent - running_recall_sent) / (batch_index + 1)

                recall_t_para = compute_recall(logits_para.view(-1,2), 
                                               batch_dict['labels'].view(-1), 
                                               batch_dict['para_mask'].view(-1))
                running_recall_para += (recall_t_para - running_recall_para) / (batch_index + 1)

                acc_t_Qtype = compute_accuracy(logits_Qtype.view(-1,2), 
                                               batch_dict['answer_type'].view(-1))
                running_acc_Qtype += (acc_t_Qtype - running_acc_Qtype) / (batch_index + 1)

                # update bar
                train_bar.set_postfix(loss=running_loss,
                                      recall_sent=running_recall_sent,
                                      recall_para=running_recall_para,
                                      acc_Qtype = running_acc_Qtype,
                                      epoch=epoch_index)
                train_bar.update()
                
                writer.add_scalar('running_loss/train', running_loss, batch_index)
                writer.add_scalar('running_recall_sent/train', running_recall_sent, batch_index)
                writer.add_scalar('running_recall_para/train', running_recall_para, batch_index)
                writer.add_scalar('running_acc_Qtype/train', running_acc_Qtype, batch_index)

            train_state['train_running_loss'].append(running_loss)
            train_state['train_running_recall_sent'].append(running_recall_sent)
            train_state['train_running_recall_para'].append(running_recall_para)
            train_state['train_running_acc_Qtype'].append(running_acc_Qtype)

            # Iterate over val dataset

            # setup: batch generator, set loss and acc to 0; set eval mode on

            dataset.set_split('val')
            batch_generator = gen_GNN_batches(dataset,
                                            batch_size=args.batch_size, 
                                            device=args.device)
            running_loss = 0.0

            running_recall_sent = 0.0
            running_recall_para = 0.0
            running_acc_Qtype = 0.0
            classifier.eval()

            for batch_index, batch_dict in enumerate(batch_generator):
                # compute the output
                with torch.no_grad():

                    logits_sent, logits_para, logits_Qtype = \
                                    classifier(batch_dict['feature_matrix'], batch_dict['adj'])

                    logits_sent = (batch_dict['sent_mask'] * logits_sent).view(-1,2)

                    labels_sent = (batch_dict['sent_mask']*batch_dict['labels'] + \
                                   batch_dict['sent_mask'].eq(0)*-100).view(-1)

                    logits_para = (batch_dict['para_mask'] * logits_para).view(-1,2)

                    labels_para = (batch_dict['para_mask']*batch_dict['labels'] + \
                                   batch_dict['para_mask'].eq(0)*-100).view(-1)


                    loss_sent = loss_func_sent(logits_sent, labels_sent) # [B,2] [B]
                    loss_para = loss_func_para(logits_para, labels_para) # [B,2] [B]
                    loss_Qtype = loss_func_Qtype(logits_Qtype.view(-1,2),
                                                 batch_dict['answer_type'].view(-1)) # [B,2] [B]

                    loss = loss_sent + loss_para + loss_Qtype
                    running_loss += (loss.item() - running_loss) / (batch_index + 1)

                    # compute the recall
                    recall_t_sent = compute_recall(logits_sent.view(-1,2), 
                                                   batch_dict['labels'].view(-1), 
                                                   batch_dict['sent_mask'].view(-1))
                    running_recall_sent += (recall_t_sent - running_recall_sent) / (batch_index + 1)

                    recall_t_para = compute_recall(logits_para.view(-1,2), 
                                                   batch_dict['labels'].view(-1), 
                                                   batch_dict['para_mask'].view(-1))
                    running_recall_para += (recall_t_para - running_recall_para) / (batch_index + 1)

                    acc_t_Qtype = compute_accuracy(logits_Qtype.view(-1,2), 
                                                   batch_dict['answer_type'].view(-1))
                    running_acc_Qtype += (acc_t_Qtype - running_acc_Qtype) / (batch_index + 1)


                val_bar.set_postfix(loss=running_loss,
                                      recall_sent=running_recall_sent,
                                      recall_para=running_recall_para,
                                      acc_Qtype = running_acc_Qtype,
                                      epoch=epoch_index)
                val_bar.update()
                                
                writer.add_scalar('running_loss/val', running_loss, epoch_index)
                writer.add_scalar('running_recall_sent/val', running_recall_sent, epoch_index)
                writer.add_scalar('running_recall_para/val', running_recall_para, epoch_index)
                writer.add_scalar('running_acc_Qtype/val', running_acc_Qtype, epoch_index)

            train_state['val_running_loss'].append(running_loss)
            train_state['val_running_recall_sent'].append(running_recall_sent)
            train_state['val_running_recall_para'].append(running_recall_para)
            train_state['val_running_acc_Qtype'].append(running_acc_Qtype)

            train_state = update_train_state(args=args, model=classifier, 
                                             optimizer = optimizer,
                                             train_state=train_state)

            scheduler.step(train_state['val_running_loss'][-1])

            train_bar.n = 0
            val_bar.n = 0
            epoch_bar.update()

            if train_state['stop_early']:
                break

except KeyboardInterrupt:
    print("Exiting loop")

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0


Exiting loop


# 保存与加载

上文已经自动保存模型.

```python
# Save checkpoint
checkpoint = {
    'model': classifier.state_dict(),
    'optimizer': optimizer.state_dict(),
    'amp': amp.state_dict()
}
torch.save(checkpoint, 'GNN_HotpotQA_amp.pt')
```

In [ ]:
from traceback import print_exc

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from tqdm.notebook import tqdm
from argparse import Namespace
import os
import ujson as json
import time

from transformers import AutoConfig, AutoModel, AutoTokenizer

from classes import QuestionNode, ParagraphTitleNode, SentenceNode, EntityNode, Adjacency_sp, auto_reload_Node
# from class_simple import *

from apex import amp

import sys

HotpotQA_path = './'
sys.path.insert(0,HotpotQA_path)

ModuleNotFoundError: No module named 'classes'

In [ ]:
# Restore
classifier = GAT_HotpotQA(features=768, hidden=32, nclass=2, dropout=0.2, alpha=0.3, nheads=8, nodes_num=450)
classifier.to('cuda')
optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()),
                      lr=1e-3)
checkpoint = torch.load('save_cache/GNN_HotpotQA.pt')

opt_level = 'O1'
classifier, optimizer = amp.initialize(classifier, optimizer, opt_level=opt_level)

classifier.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])
amp.load_state_dict(checkpoint['amp'])

# Continue training

# 阶段2

## find top K sents

In [56]:
dataset = HotpotQA_Dataset_New.build_dataset(i_from = 0,i_to = 10)
dataset.set_parameters(450, 0)

In [57]:
for batch_dict in gen_batches(dataset,
                                batch_size=7, 
                                device='cuda'):
    break

In [58]:
batch_dict.keys()

dict_keys(['feature_matrix', 'adj', 'sent_mask', 'para_mask', 'labels', 'answer_type', 'ans_yes_no', 'ques_tokens', 'answer_tokens', 'sent_tokens'])

In [ ]:
def find_ans_spans(target, tokens, offets_type = 'position', top_num = None):
    assert offets_type in ['position', 'range'] and \
        type(target) == type(tokens) == list
    len_x1 = len(target)
    len_x2 = len(tokens)
    if len_x1 == 0 or len_x2 == 0 or len_x1 > len_x2:
        return [0,0]
    
    i1=0
    i2=0
    i2_current = 0
    spans = []
    while i2 <= len_x2 - len_x1:
        if top_num and len(spans) == top_num:
            break
        i2_current = i2
        while i1 < len_x1:
            if target[i1] != tokens[i2]: 
                i1 = 0
                i2 = i2_current + 1
                break
            else:
                i1 += 1
                i2 += 1
                
        if not i1 < len_x1:
            i1 = 0
            if offets_type == 'position':  
                spans.append([i2_current, i2_current+len_x1-1])
            else:
                spans.append([i2_current, i2_current+len_x1])
                
    return spans[:top_num+1] if top_num else spans[:]

In [ ]:
def convert_batch_to_LM(classifier, batch_dict, topN = 4,
                        sep_token = '[SEP]', cls_token = '[CLS]'):
    '''convert batch from GNN to LM model(QA fine-tuning)'''
    batch_dict_for_LM = []
    logits_sent, _, __ = classifier(batch_dict['feature_matrix'], batch_dict['adj'])
    topN_sents = []
    for batch_index in range(batch_dict['sent_mask'].shape[0]):
        sent_index = torch.nonzero(batch_dict['sent_mask'][batch_index].view(-1,).long()).flatten()
        sent_logits = torch.index_select(logits_sent[batch_index], 0, sent_index)
        sent_logits_soft = F.softmax(sent_logits, dim = -1).cpu()
        value, _index = torch.max(sent_logits_soft, dim=-1)
        sent_scores = value.tolist()
        sent_score_index = [(score, index) for score, index in zip(sent_scores, range(len(sent_scores)))]
        sent_score_index_sort = sorted(sent_score_index,key=lambda x:x[0], reverse=True)
        sent_indexes = [x[-1] for x in sent_score_index_sort[:topN]]
        topN_sents.append(sent_indexes)
        
    batch_sent_tokens = []
    for batch_index, sents in enumerate(topN_sents):
        temp = []
        for i_sent in sents: temp.extend(batch_dict['sent_tokens'][batch_index][i_sent])
        batch_sent_tokens.append(temp)

    batch_ques_sent_tokens = [[sep_token] + batch_dict['ques_tokens'] + sep_token + \
                              sent_tokens + cls_token for sent_tokens in batch_sent_tokens]
    

topN_sents_one_batch(batch_dict,logits_sent)

In [17]:
sent_index = torch.nonzero(batch_dict['sent_mask'][0].view(-1,).long()).flatten()

In [18]:
sent_index.to('cpu')

tensor([  2,  10,  14,  22,  31,  33,  36,  43,  49,  54,  58,  66,  72,  74,
         77,  79,  81,  85,  89,  93,  97, 101, 105, 106, 108, 112, 117, 121,
        123, 127, 130, 134, 138, 143, 148, 151, 157])

In [19]:
sent_index.shape

torch.Size([37])

In [20]:
len(batch_dict['sent_tokens'][0])

37

In [21]:
sent_logits = torch.index_select(logits_sent[0], 0, sent_index)
sent_logits_soft = F.softmax(sent_logits, dim = -1).cpu()
sent_logits_soft.shape

torch.Size([37, 2])

In [22]:
value, _index = torch.max(sent_logits_soft, dim=-1)
sent_scores = value.cpu().tolist()
len(sent_scores)

37

In [23]:
sent_score_index = [(score, index) for score, index in zip(sent_scores, range(len(sent_scores)))]

In [24]:
sent_score_index_sort = sorted(sent_score_index,key=lambda x:x[0],reverse=True)
sent_indexes = [x[-1] for x in sent_score_index_sort[:4]]
sent_indexes
# 获取batch中一个元素的前4句.

[1, 14, 12, 16]

## 测试find span

In [66]:
sent_tokens = batch_dict['sent_tokens']

sent_final_tokens = []
for i in sent_indexes: sent_final_tokens.extend(sent_tokens[0][i])
print(sent_final_tokens)

['▁It', '▁was', '▁founded', '▁in', '▁Pine', '▁Bluff', ',', '▁Arkansas', '▁in', '▁1986', '.', '▁Cardinal', '▁Health', '▁also', '▁owns', '▁the', '▁franchise', '▁chain', '▁The', '▁Medicine', '▁Shop', 'pe', '.', '▁Leader', '▁Drug', '▁Store', 's', '▁is', '▁a', '▁network', '▁of', '▁over', '▁3', ',', '100', '▁independently', '▁owned', '▁and', '▁operated', '▁pharmacies', '.', '▁Gray', '▁Drug', '▁was', '▁a', '▁drug', 'store', '▁chain', '▁in', '▁Cleveland', ',', '▁Ohio', '.']


In [67]:
answer_tokens_test =  ['▁network', '▁of', '▁over']

In [68]:
answer_tokens_test in sent_final_tokens

False

In [82]:
print(answer_tokens_test)
print(sent_final_tokens[29:32])

['▁network', '▁of', '▁over']
['▁network', '▁of']


In [86]:
def find_ans_spans(target, tokens, offets_type = 'position', top_num = None):
    assert offets_type in ['position', 'range'] and \
        type(target) == type(tokens) == list
    len_x1 = len(target)
    len_x2 = len(tokens)
    if len_x1 == 0 or len_x2 == 0 or len_x1 > len_x2:
        return [0,0]
    
    i1=0
    i2=0
    i2_current = 0
    spans = []
    while i2 <= len_x2 - len_x1:
        if top_num and len(spans) == top_num:
            break
        i2_current = i2
        while i1 < len_x1:
            if target[i1] != tokens[i2]: 
                i1 = 0
                i2 = i2_current + 1
                break
            else:
                i1 += 1
                i2 += 1
                
        if not i1 < len_x1:
            i1 = 0
            if offets_type == 'position':  
                spans.append([i2_current, i2_current+len_x1-1])
            else:
                spans.append([i2_current, i2_current+len_x1])
                
    return spans[:top_num+1] if top_num else spans[:]

In [87]:
find_ans_spans(answer_tokens_test, sent_final_tokens)

[[29, 31]]

## 计算span

将`ques_feature`和`content_features`加上`[SEP] [CLS]`拼接在一起.

In [88]:
answer_tokens_test2 = [ 'Australian', 'GT', 'Championship',]

In [89]:
sent_tokens_combine = []
for s_index in sent_indexes: sent_tokens_combine.extend(sent_tokens[0][s_index])

In [90]:
final_tokens = ['<x1>','[ques]','<x2>'] + sent_tokens_combine + ['<cls>']
ans_spans = find_ans_spans(answer_tokens_test2, final_tokens, top_num = 2)
ans_spans

[]

In [76]:
len(final_tokens)

57

# top K sent acc

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
logits = torch.tensor([[[0.2, 0.8],
                      [0.3,0.7],
                      [0.9,0.1]],
                      [[0.2, 0.8],
                      [0.6,0.4],
                      [0.99,0.01]]])

In [23]:
value, indexs = logits.max(dim=-1)

In [31]:
values2,indexs2 = value[0].topk(2, dim=0)

In [33]:
predict = torch.tensor([[0,1,1],[1,0,0]])
torch.index_select(predict[0], 0, indexs2)

tensor([1, 0])